In [1]:
########################################
# ライブラリ
########################################

import jax
import jax.numpy as jnp
from jax import nn
from jax.nn.initializers import glorot_normal, normal

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from tqdm import trange as tqdm_range

In [2]:
########################################
# ニューラルネットワーク（フィードフォワード）
########################################

@jax.jit
def Linear(params, x):
    return jnp.dot(x, params["W"]) + params["b"]


@jax.jit
def ANN(params, x):
    y1 = Linear(params["linear1"], x)
    z1 = nn.relu(y1)
    y2 = Linear(params["linear2"], z1)
    z2 = nn.softmax(y2)
    return z2

In [3]:
########################################
# 訓練
########################################

# クロスエントロピー誤差
@jax.jit
def cross_entropy_loss(params, X, y):
    logits = ANN(params, X)
    return jnp.mean(-jnp.sum(y * jnp.log(logits), axis=1))

# パラメータの更新
@jax.jit
def update_params(params, grad, lr = 0.01):
    params["linear1"]["W"] -= lr * grad["linear1"]["W"]
    params["linear2"]["W"] -= lr * grad["linear2"]["W"]
    params["linear1"]["b"] -= lr * grad["linear1"]["b"]
    params["linear2"]["b"] -= lr * grad["linear2"]["b"]
    return params

# 与えられたXとyで勾配を計算&更新
@jax.jit
def train(params, X, y):
    grad = jax.grad(cross_entropy_loss)(params, X, y)
    return update_params(params, grad)

# バッチ毎に訓練
@jax.jit
def train_for_each_batch(batch_idx, params):
    target_train_indices = jax.lax.dynamic_slice(index, [batch_idx*batch_size], [batch_size])
    params = train(params, X_train[target_train_indices], y_train[target_train_indices])
    return params

In [4]:
########################################
# データセットの読み込み
########################################

iris_dataset = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_dataset['data'], iris_dataset['target'], test_size=0.25,  random_state=0)
X_train, X_test, y_train, y_test = jax.device_put(X_train), jax.device_put(X_test), jax.device_put(y_train), jax.device_put(y_test)
y_train = jnp.eye(3)[y_train]
y_test = jnp.eye(3)[y_test]

In [8]:
########################################
# 学習してみる
########################################

# パラメータの初期化
rng = jax.random.PRNGKey(0)
rng1, rng2 = jax.random.split(rng)
rng1w, rng1b = jax.random.split(rng1)
rng2w, rng2b = jax.random.split(rng2)

params = {
    "linear1": {
        "W": glorot_normal()(rng1w, (4, 100)),
        "b": normal()(rng1b, (100,))
    },
    "linear2": {
        "W": glorot_normal()(rng2w, (100, 3)),
        "b": normal()(rng2b, (3,))
    }
}

# バッチサイズ
batch_size = 50
# エポック数
epoch_nums = 100

# 訓練を回す
for epoch_id in range(epoch_nums):

    key = jax.random.PRNGKey(epoch_id+1)
    
    # 訓練データのインデックスをシャッフル
    index = jax.random.permutation(key, X_train.shape[0])
    # バッチ数
    batch_length = jnp.ceil(X_train.shape[0] / batch_size)

    # バッチ毎にパラメータを更新していく
    params = jax.lax.fori_loop(0, int(batch_length), train_for_each_batch, params)

    # 誤差の確認
    print(
        "訓練誤差:",
        '{:.3f}'.format(cross_entropy_loss(params, X_train, y_train)),
        "汎化誤差:",
        '{:.3f}'.format(cross_entropy_loss(params, X_test, y_test)),
        f"【 Epoch: {epoch_id} / {epoch_nums} 】"
    )

訓練誤差: 1.332 汎化誤差: 1.460 【 Epoch: 0 / 100 】
訓練誤差: 1.089 汎化誤差: 1.164 【 Epoch: 1 / 100 】
訓練誤差: 0.982 汎化誤差: 1.047 【 Epoch: 2 / 100 】
訓練誤差: 0.923 汎化誤差: 0.984 【 Epoch: 3 / 100 】
訓練誤差: 0.878 汎化誤差: 0.939 【 Epoch: 4 / 100 】
訓練誤差: 0.838 汎化誤差: 0.900 【 Epoch: 5 / 100 】
訓練誤差: 0.805 汎化誤差: 0.866 【 Epoch: 6 / 100 】
訓練誤差: 0.776 汎化誤差: 0.837 【 Epoch: 7 / 100 】
訓練誤差: 0.750 汎化誤差: 0.811 【 Epoch: 8 / 100 】
訓練誤差: 0.727 汎化誤差: 0.788 【 Epoch: 9 / 100 】
訓練誤差: 0.707 汎化誤差: 0.768 【 Epoch: 10 / 100 】
訓練誤差: 0.690 汎化誤差: 0.750 【 Epoch: 11 / 100 】
訓練誤差: 0.674 汎化誤差: 0.733 【 Epoch: 12 / 100 】
訓練誤差: 0.659 汎化誤差: 0.718 【 Epoch: 13 / 100 】
訓練誤差: 0.646 汎化誤差: 0.705 【 Epoch: 14 / 100 】
訓練誤差: 0.634 汎化誤差: 0.692 【 Epoch: 15 / 100 】
訓練誤差: 0.622 汎化誤差: 0.680 【 Epoch: 16 / 100 】
訓練誤差: 0.611 汎化誤差: 0.669 【 Epoch: 17 / 100 】
訓練誤差: 0.601 汎化誤差: 0.659 【 Epoch: 18 / 100 】
訓練誤差: 0.591 汎化誤差: 0.649 【 Epoch: 19 / 100 】
訓練誤差: 0.582 汎化誤差: 0.639 【 Epoch: 20 / 100 】
訓練誤差: 0.573 汎化誤差: 0.631 【 Epoch: 21 / 100 】
訓練誤差: 0.565 汎化誤差: 0.622 【 Epoch: 22 / 100 